In [202]:
import numpy as np
import cv2


In [203]:
img = cv2.imread('lena.bmp', 0)

# Downsampling lena from 512x512 to 64x64
> This part is the same as the previous homework 6

1. binarize lena image as HW2
2. use 8x8 blocks as a unit
3. take top-most left pixel as downsampled data

Note: Result is a 64x64 matrix

## Step 1: Binarize the image

In [204]:
binarized_img = np.zeros(img.shape, np.int8)
for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        if img[i][j] >= 128:
            binarized_img[i][j] = 1

## Step 2, 3: Downsample the image

First we initialize `downsampled_img` as a 64x64 matrix, with all elements being 0.

Then we iterate through each 8x8 block in the binarized image, and take the top-most left pixel as the downsampled data.

In [205]:
downsampled_img  = np.zeros((64, 64), np.int8)
for i in range(0, 64):
    for j in range(0, 64):
        downsampled_img[i][j] = binarized_img[i*8][j*8]

# Create marked image

## Yokoi operator

> This Yokoi operator is implemented by adding little modifications from the previous homework 6.  
> $\rightarrow$ For more information (e.g. the definition of `h` and `f` functions, and the definition of Yokoi connectivity number) please refer to the previous homework 6.

### primitive functions

In [206]:
def h(b, c, d, e):
    if b == c and (d != b or e != b):
        return 'q'
    elif b == c and (d == b and e == b):
        return 'r'
    else:
        return 's'

In [207]:
def Yokoi_f(a_1, a_2, a_3, a_4):
    if a_1 == a_2 and a_2 == a_3 and a_3 == a_4 and a_4 == 'r':
        return 5
    else:
        connectivity_number = 0
        for a_i in [a_1, a_2, a_3, a_4]: 
            if a_i == 'q':
                connectivity_number += 1
        return connectivity_number

For the Yokoi_operator function, we call the primitive functions above and do the following steps:  

1. Initialize a matrix `Yokoi_result`with all elements being 0, having the same size as the downsampled image.  
2. Iterate through each pixel in the downsampled image, and apply the Yokoi operator to determine the connectivity number.  
3. Update the matrix with the connectivity number.  

In [208]:
def Yokoi_operator(img):
    Yokoi_result = np.zeros(img.shape, np.int8)
    rows, cols = img.shape
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if img[i][j] == 1:
                x0 = img[i][j]
                x1 = img[i][j+1] if j < cols-1 else 0
                x2 = img[i-1][j] if i > 0 else 0
                x3 = img[i][j-1] if j > 0 else 0
                x4 = img[i+1][j] if i < rows-1 else 0
                x5 = img[i+1][j+1] if i < rows-1 and j < cols-1 else 0
                x6 = img[i-1][j+1] if i > 0 and j < cols-1 else 0
                x7 = img[i-1][j-1] if i > 0 and j > 0 else 0
                x8 = img[i+1][j-1] if i < rows-1 and j > 0 else 0

                a1 = h(x0, x1, x6, x2)
                a2 = h(x0, x2, x7, x3)
                a3 = h(x0, x3, x8, x4)
                a4 = h(x0, x4, x5, x1)

                connectivity_number = Yokoi_f(a1, a2, a3, a4)
                Yokoi_result[i][j] = connectivity_number
    return Yokoi_result


## Pair relationship operator

### primitive functions

Note: To avoid using the same name as the primitive function `h` in the Yokoi operator, we name this function as `pair_h`.

In [209]:
def pair_h(a, m):
    if a == m:
        return 1
    else:
        return 0

In [210]:
def output(x0, x1, x2, x3, x4, m):
    if pair_h(x0, m) == 1 and pair_h(x1, m) + pair_h(x2, m) + pair_h(x3, m) + pair_h(x4, m) >= 1:
        return 'p'
    else:
        return 'q'


In [211]:
def pair_relationship_operator(Yokoi_result, m):
    pair_result = np.zeros(Yokoi_result.shape, dtype = 'U1')
    rows, cols = Yokoi_result.shape
    for i in range(rows):
        for j in range(cols):
            if Yokoi_result[i][j] == 1:
                right = Yokoi_result[i][j+1] if j < cols-1 else 0
                top = Yokoi_result[i-1][j] if i > 0 else 0
                left = Yokoi_result[i][j-1] if j > 0 else 0
                bottom = Yokoi_result[i+1][j] if i < rows-1 else 0
                pair_result[i][j] = output(Yokoi_result[i][j], right, top, left, bottom, m)
            else:
                pair_result[i][j] = ' '

    return pair_result

# connected shrink operator

In [212]:
def connected_shrink_h(b, c, d, e):
    if b == c and (d != b or e != b):
        return 1
    return 0

def connected_shrink_operator(img, i, j):
    rows, cols = img.shape
    x0 = img[i][j]
    x1 = img[i][j+1] if j < cols-1 else 0
    x2 = img[i-1][j] if i > 0 else 0
    x3 = img[i][j-1] if j > 0 else 0
    x4 = img[i+1][j] if i < rows-1 else 0
    x5 = img[i+1][j+1] if i < rows-1 and j < cols-1 else 0
    x6 = img[i-1][j+1] if i > 0 and j < cols-1 else 0
    x7 = img[i-1][j-1] if i > 0 and j > 0 else 0
    x8 = img[i+1][j-1] if i < rows-1 and j > 0 else 0

    a1 = connected_shrink_h(x0, x1, x6, x2)
    a2 = connected_shrink_h(x0, x2, x7, x3)
    a3 = connected_shrink_h(x0, x3, x8, x4)
    a4 = connected_shrink_h(x0, x4, x5, x1)

    if sum([a1, a2, a3, a4]) == 1:
        return 0  
    return 1  


'\n'

# thinning operator

In [213]:
def thinning_operator(img, pair_result):
    rows, cols = img.shape
    for i in range(rows):
        for j in range(cols):
            if pair_result[i][j] == 'p':
                img[i][j] = connected_shrink_operator(img, i, j)
    return img

def iterative_thinning(downsampled_img):
    previous_result = None
    current_result = downsampled_img.copy()
    iteration = 0
    
    while True:
        iteration += 1
        previous_result = current_result.copy()
        
        Yokoi_result = Yokoi_operator(current_result)
        pair_result = pair_relationship_operator(Yokoi_result, 1)
        current_result = thinning_operator(current_result, pair_result)
        
        if np.array_equal(previous_result, current_result):
            print(f"Converged after {iteration} iterations")
            break
        
        if iteration > 100:  # Safety limit
            print("Reached maximum iterations")
            break
    
    return current_result

# Main functinon

In this part, we follow the following steps:  

1. Create the marked image by applying Yokoi operator, pair relationship operator.
    - Yokoi operator result: `Yokoi_result`
    - Pair relationship operator result: `pair_result`


In [214]:
Yokoi_result = Yokoi_operator(downsampled_img)
np.savetxt('yokoi_result.txt', Yokoi_result, fmt='%d', delimiter=' ')

pair_result = pair_relationship_operator(Yokoi_result, 1)
np.savetxt('pair_result.txt', pair_result, fmt='%s', delimiter=' ')

thinning_result = iterative_thinning(downsampled_img)
np.savetxt('thinning_result.txt', thinning_result, fmt='%d', delimiter=' ')

display_image = (thinning_result * 255).astype(np.uint8)
cv2.imwrite('thinned_lena.bmp', display_image)

Converged after 7 iterations


True